In [18]:
!pip install numpy matplotlib opencv-python

In [21]:
import cv2
import numpy as np
import glob

# Definir dimensiones del tablero (esquinas internas)
chessboard_size = (9, 6)

# Criterio de terminación de subpixeles
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Preparar puntos 3D (0,0,0 ... 8,5,0)
objp = np.zeros((np.prod(chessboard_size), 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

objpoints = [] # Puntos 3D
imgpoints = [] # Puntos 2D

# Cargar imágenes del tablero
images = glob.glob('chessboard/*.jpeg')

for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    if ret:
        objpoints.append(objp)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
        imgpoints.append(corners2)

# Calibración
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Matriz de la cámara:\n", mtx)
print("Coeficientes de distorsión:\n", dist)

# Corrección de una imagen de prueba
img = cv2.imread(images[0])
h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
cv2.imwrite('calibrated_result.jpg', dst)

Matriz de la cámara:
 [[769.7116203    0.         378.44990516]
 [  0.         765.11004269 500.59406819]
 [  0.           0.           1.        ]]
Coeficientes de distorsión:
 [[ 5.74602318e-02 -3.76518225e-01 -1.85796848e-04  1.59707834e-03
   6.28075498e-01]]


True

In [22]:
# Calcular error de reproyección
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error

print("Error medio de reproyección:", mean_error/len(objpoints))

Error medio de reproyección: 0.05039528624589758


In [23]:
# Video en tiempo real con corrección de distorsión
cap = cv2.VideoCapture(0)  # Usar cámara web

print("Presiona 'q' para salir del video")
print("Presiona 's' para guardar un frame corregido")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Aplicar corrección
    h, w = frame.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
    undistorted = cv2.undistort(frame, mtx, dist, None, newcameramtx)
    
    # Mostrar lado a lado
    combined = np.hstack((frame, undistorted))
    combined = cv2.resize(combined, (1200, 400))  # Redimensionar para pantalla
    
    cv2.putText(combined, 'Original', (50, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(combined, 'Corregida', (650, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    
    cv2.imshow('Comparacion: Original vs Corregida', combined)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        cv2.imwrite('frame_comparison.jpg', combined)
        print("Frame guardado como 'frame_comparison.jpg'")

cap.release()
cv2.destroyAllWindows()

Presiona 'q' para salir del video
Presiona 's' para guardar un frame corregido


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/brosgor/Documents/miGit/computacion-visual/luis/practica1/.venv/lib/python3.13/site-packages/cv2/qt/plugins"


KeyboardInterrupt: 